In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import json
import time
from tqdm import tqdm
from collections import Counter
from datetime import datetime, timezone

import pandas as pd

import os
# os.environ["BRT_ENV"] = 'prod'
os.environ['AWS_PROFILE'] = 'sso'
import brtdevkit
brtdevkit.log = 'info'
print(brtdevkit.__version__)

from brtdevkit.core.db.athena import AthenaClient, Table
from brtdevkit.data import Image, Dataset

import sys
sys.path.append('/home/bluerivertech/li.yu/code/JupiterCVML/europa/base/src/europa')
from aletheia_dataset_creator.dataset_tools.aletheia_dataset_helpers import imageids_to_dataset, imageids_to_dataframe, get_stereo_images_df, imageids_to_annotation_df

7.6.1


### get dataset

In [3]:
#if the dataset is on aletheia you can retrieve it or read in the csv
test_dataset = Dataset.retrieve(id="64cad86b14aabd709586690d")
# test_dataset = Dataset.retrieve(name="Jupiter_human_and_vehicle_train_dataset_v1")
test_df = test_dataset.to_dataframe()
print(test_df.shape)
test_df.head(2)

(31978, 111)


,artifact_debayeredrgb_0__id,artifact_debayeredrgb_0_content_hash,artifact_debayeredrgb_0_created_at,artifact_debayeredrgb_0_data_category,artifact_debayeredrgb_0_id,artifact_debayeredrgb_0_image,artifact_debayeredrgb_0_is_archived,artifact_debayeredrgb_0_kind,artifact_debayeredrgb_0_project_name,artifact_debayeredrgb_0_s3_bucket,...,system_name,teleop_request,teleop_response,terrain_type,updated_at,vpu_position,was_compressed,water_management,weather,weather_summary
0,6462816d000a1353b5968ce6,628a60d1e2e326fcdad9a0a0877a0726,2023-05-15T19:01:01.895000,eng,6462816d000a1353b5968ce6,6462816d000a1353b5968ce4,False,debayeredrgb,jupiter,brt-mesa-jupiter-images-eng,...,loamy_731,NaN,NaN,openfield,2023-06-28T17:06:00.651000,1,False,tile,sunny,"{'datetime': '01:00:00', 'datetimeEpoch': 1683..."
1,64628181c458bb640eaece42,9645ce704febb05f830e55166b429a9e,2023-05-15T19:01:21.622000,eng,64628181c458bb640eaece42,64628181c458bb640eaece40,False,debayeredrgb,jupiter,brt-mesa-jupiter-images-eng,...,loamy_731,NaN,NaN,openfield,2023-06-28T17:06:00.650000,1,False,tile,sunny,"{'datetime': '01:00:00', 'datetimeEpoch': 1683..."


In [5]:
test_df = test_df[test_df.camera_location.str.endswith('left')]
# test_df = test_df[test_df.camera_location.str.startswith('front')]
test_df.shape

(30000, 109)

In [3]:
test_df[['id', 'camera_location']].groupby('camera_location').count()

,id
camera_location,
front-center-left,4912
front-left-left,4844
front-right-left,4862
rear-left,5830
side-left-left,5741
side-right-left,5789


In [4]:
# test_df = test_df[test_df.camera_location == 'rear-left']
test_df['datehm'] = test_df.collected_on.apply(lambda x:str(x)[:16])
l = list(test_df['datehm'].unique())
l.sort()
for t in l:
    print(t, len(test_df[test_df.datehm == t]))

2023-04-27T12:55 75
2023-04-27T12:56 458
2023-04-27T12:57 572
2023-04-27T12:58 524
2023-04-27T12:59 361
2023-04-27T13:00 431
2023-04-27T13:01 35
2023-04-27T13:02 543
2023-04-27T13:03 11
2023-05-01T22:50 277
2023-05-01T22:51 322
2023-05-01T22:52 242
2023-05-01T22:53 392
2023-05-01T22:54 535
2023-05-01T22:55 461
2023-05-01T22:56 380
2023-05-01T22:57 67
2023-05-01T22:58 386
2023-05-01T22:59 394
2023-05-01T23:00 477
2023-05-01T23:01 531
2023-05-01T23:02 542
2023-05-01T23:03 542
2023-05-01T23:04 528
2023-05-01T23:05 332
2023-05-01T23:06 273
2023-05-01T23:07 498
2023-05-01T23:08 547
2023-05-01T23:09 34
2023-05-04T00:48 321
2023-05-04T00:49 525
2023-05-04T00:50 506
2023-05-04T00:51 291
2023-05-04T00:52 430
2023-05-04T00:53 514
2023-05-04T00:54 93
2023-05-04T00:55 118
2023-05-04T00:56 280
2023-05-04T00:57 303
2023-05-04T00:58 520
2023-05-04T00:59 509
2023-05-04T01:00 183
2023-05-04T01:01 448
2023-05-04T01:02 550
2023-05-04T01:03 458
2023-05-04T01:04 499
2023-05-04T01:05 535
2023-05-04T01:06 26

In [19]:
# test_dfs = []
test_dfs.append(test_df)
test_df = pd.concat(test_dfs, ignore_index=True)
test_df.shape

(23488, 95)

### Run query

In [8]:
# hard_drive_name = 'JUPD-048_2022-09-27'
# start_datetime  = datetime(2022, 9, 28, hour=1, minute=18, second=0) #, tzinfo=timezone.utc)
# end_datetime  = datetime(2022, 9, 28, hour=2, minute=47, second=0) #, tzinfo=timezone.utc)
# start_datetime  = datetime(2022, 9, 28, tzinfo=timezone.utc) #, hour=3, minute=33, second=33) #, tzinfo=timezone.utc)
start_datetime  = datetime(2023, 8, 28, hour=0, minute=0)   #, tzinfo=timezone.utc)
end_datetime    = datetime(2023, 8, 30, hour=23, minute=59)  # this is EXclusive

database = "mesa-data-catalog-prod"
table = "image_jupiter"

query = \
f"""
SELECT *
FROM {table} T
WHERE 
    T.collected_on >= cast('{start_datetime}' as timestamp)
    AND T.collected_on < cast('{end_datetime}' as timestamp)
    AND T.robot_name = 'loamy_06'
"""

"""
SELECT
        *
    FROM annotation_jupiter
    WHERE
     annotation_jupiter.image IN ('642607df1620247e2285d6ac')
"""

"""
SELECT id, camera_location, operation_time, collected_on, state, county, operating_field_name, hdr_mode, robot_name, special_notes
T.collected_on >= cast('{start_datetime}' as timestamp)
    AND T.collected_on < cast('{end_datetime}' as timestamp)
    AND T.camera_location LIKE 'front-%'
    AND T.camera_location LIKE '%-left'
    AND T.hard_drive_name = 'JUPD-154_2023-01-14'
    AND T.robot_name LIKE 'loamy%'
    AND T.operation_time != 'daytime'
    AND T.has_nearby_stop_event = true
ORDER BY RAND()
LIMIT 40000
"""

start = time.time()
athena = AthenaClient()
df = athena.get_df(query)
end = time.time()
print(end - start, 's')
df.shape, len(df) // 2

8.189175367355347 s


((0, 212), 0)

In [3]:
df['datehour'] = df.collected_on.apply(lambda x:str(x)[:13])
# df['datetime'] = df.collected_on.apply(datetime.fromisoformat)

<ipython-input-3-3a11d5402787>:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df['datehour'] = df.collected_on.apply(lambda x:str(x)[:13])


In [5]:
df[['id', 'collected_on', 'datehour']]

,id,collected_on,datehour
0,627ee633ced080bd315fc03d,2022-03-30 23:27:41.731,2022-03-30 23
1,627ee69f68e3fb073f05a636,2022-03-30 23:14:13.278,2022-03-30 23
2,627ee60d23b056f3b1206f12,2022-03-30 23:23:48.548,2022-03-30 23
3,627edff068e3fb073f058c6b,2022-03-30 23:39:01.724,2022-03-30 23
4,62801ab1ec0a49ecdd35b458,2022-03-30 23:29:03.751,2022-03-30 23
...,...,...,...
27545,627edb26dc1484bdb69fde82,2022-03-30 23:40:54.394,2022-03-30 23
27546,627edc9b66683987acd0a61b,2022-03-30 23:40:10.556,2022-03-30 23
27547,62801bf0ecd98b51744077ef,2022-03-30 23:29:05.180,2022-03-30 23
27548,627edac8ec0a49ecdd317ad0,2022-03-30 23:40:08.295,2022-03-30 23


In [12]:
l = df.sample(5).id.to_list()
for i in l:
    print(i)

63cfa3b78ef9793417505084
640f8760345d1f8f3ef77e11
63c7606045ddf17c18f39823
63bfa4e182220558192d9ca4
63c748d2cbea581aac0d84ac


In [9]:
# dfs = []
dfs.append(df)

In [10]:
df = pd.concat(dfs, ignore_index=True)
df.shape

(96341, 184)

#### Load ids from labelbox exports

In [12]:
json_path = '/home/bluerivertech/li.yu/code/notebooks/dust_head/export-2023-08-03T08_39_11.733Z.json'
with open(json_path, 'r') as f:
    data = json.load(f)
print(len(data))
ids = [item['Labeled Data'][61:85] for item in data]
print(len(ids))

4913
4913


In [16]:
csv_path = '/home/bluerivertech/li.yu/code/notebooks/datasets/train_v5.12/driveable_not_labeled.csv'
csv_path2 = '/home/bluerivertech/li.yu/code/notebooks/datasets/train_v5.12/less_than_half_labeled.csv'
ids = pd.read_csv(csv_path).id.to_list() + pd.read_csv(csv_path2).id.to_list()
ids = list(set(ids))
len(ids)

2628

### Create dataset without labels

In [6]:
# Dataset.create(
#     name='Jupiter_2023_04_05_loamy869_dust_collection_stereo',
#     description="stereo data for Jupiter_2023_04_05_loamy869_dust_collection, 27K image pairs",
#     kind=Dataset.KIND_IMAGE,
#     mode='stereo',
#     image_ids=test_df.id.to_list())

imageids_to_dataset(
    image_ids=test_df.id.to_list(),
    dataset_name='Jupiter_2023_may_loamy731_vehicle_dust_human_stereo_part2',
    dataset_description='second half of 3rd batch of vehicle dust collection, collected on April 27th, May 1st and 4th, 32K images',
#     dataset_kind=Dataset.KIND_ANNOTATION,
    dataset_kind=Dataset.KIND_IMAGE,
    mode='stereo')

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Size of stereo dataframe: 31978


### Create dataset with labels

In [6]:
# df = pd.read_csv('/home/bluerivertech/li.yu/data/Jupiter_2023_03_10_851_dust_human/dust_for_training.csv')
# df.shape

((1500, 1), (400, 1), (1900, 1))

In [5]:
imageids_to_dataset(
    image_ids=test_df.id.to_list(),
    dataset_name='Jupiter_2023_may_loamy731_vehicle_dust_human_stereo_labeled_part2_v2',
    dataset_description='7K images partially labeled out of Jupiter_2023_may_loamy731_vehicle_dust_human_stereo_part2',
    dataset_kind=Dataset.KIND_ANNOTATION,
#     dataset_kind=Dataset.KIND_IMAGE,
#     production_dataset=False,  # use with partial labels ?
    mode='stereo')

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Filtered 31226 annotations.
Warning 31978 images do not have a corresponding annotation.
INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


ValueError: Number of processes must be at least 1

### Prune df to get all stereo pairs

In [10]:
df = test_df
g = df.groupby('group_id')
print(len(g))

unmatched_image_ids = []
for group_id, group in tqdm(g, total=len(g)):
    camera_locations = group.camera_location.to_list()
    pure_cameras = set()
    for c in camera_locations:
        if c.endswith('left'):
            pure_cameras.add(c[:-5])
        else:
            pure_cameras.add(c[:-6])
    for c in pure_cameras:
        cdf = group[group.camera_location.str.startswith(c)]
        if len(cdf) % 2 != 0:
            unmatched_image_ids += cdf.id_img_left.to_list()
print(len(unmatched_image_ids))

stereo_df = df[~df.id.isin(unmatched_image_ids)]
stereo_df.shape, df.shape

  0%|          | 1/9519 [00:00<27:00,  5.87it/s]

9519


100%|██████████| 9519/9519 [00:06<00:00, 1361.97it/s]


0


((54808, 109), (54808, 109))

### Retrieve stereo pairs given left ids

In [6]:
left_df = test_df
stereo_df = get_stereo_images_df(left_df)
print(left_df.shape, stereo_df.shape)

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Size of stereo dataframe: 27404
(54808, 111) (27404, 112)


### Retrieve stereo pairs give left ids and raw stereo df

In [90]:
df = df_sample
df.shape

(99998, 165)

In [93]:
mdf = pd.concat(dfs, ignore_index=True)
mdf.shape

(3951312, 165)

In [94]:
df['image_id'] = df['id']
raw_stereo_df = mdf
raw_stereo_df['image_id'] = raw_stereo_df['id']

sub_df = raw_stereo_df[raw_stereo_df.image_id.isin(df.image_id)]
print(sub_df.shape)
sub_df = raw_stereo_df[raw_stereo_df.group_id.isin(sub_df.group_id)]
print(sub_df.shape)

(99998, 166)
(1040976, 166)


In [97]:
from tqdm import tqdm

In [98]:
single_image_ids = df.image_id.to_list()
stereo_image_ids = single_image_ids.copy()
for image_id in tqdm(single_image_ids, total=len(single_image_ids)):
    camera_location = df[df.image_id == image_id].iloc[0].camera_location
    group_id = sub_df[sub_df.image_id == image_id].iloc[0].group_id
    stereo_locations = sub_df[sub_df.group_id == group_id].camera_location.to_list()
    other_location = camera_location[:-4]+'right' if camera_location.endswith('left') else camera_location[:-5]+'left'
    assert other_location in stereo_locations
    other_image_id = sub_df[(sub_df.group_id == group_id) & (sub_df.camera_location == other_location)].iloc[0].image_id
    stereo_image_ids.append(other_image_id)
print(len(single_image_ids), len(set(stereo_image_ids)))

 20%|███████████████████████                                                                                          | 20355/99998 [1:50:37<7:12:51,  3.07it/s]


KeyboardInterrupt: 

## Download dataset

In [2]:
dataset_name = 'dust_metadata_labels'
dataset_dir = os.path.join('/home/bluerivertech/li.yu/data', dataset_name)
os.makedirs(dataset_dir, exist_ok=True)

test_dataset = Dataset.retrieve(name=dataset_name)
test_df = test_dataset.to_dataframe()
test_dataset.download(dataset_dir, df=test_df)

2023-03-14 19:34:12,897 - Dataset - INFO - start downloading dataset: 640bc1e3605dc17c12a63162 into /home/bluerivertech/li.yu/data/Jupiter_2023_03_10_851_dust_human
100%|██████████| 79990/79990 [02:16<00:00, 585.94it/s]
2023-03-14 19:36:45,185 - Dataset - INFO - finished downloading dataset: 640bc1e3605dc17c12a63162
